In [6]:
# Imports
import glob
import math
import ntpath
import os
import shutil
from pathlib import Path

import numpy as np
import pandas as pd

import mne

from mne.channels import read_custom_montage
from mne.epochs import concatenate_epochs

from mne import Epochs, pick_types, find_events, EpochsArray,create_info
from mne.io import concatenate_raws, read_raw_edf

from mne.preprocessing import (ICA, create_eog_epochs, create_ecg_epochs, corrmap)


# Loading the Data

In [7]:
data_path = Path(mne.datasets.sample.data_path(verbose=False))
sample_dir = data_path / 'MEG' / 'sample'
subjects_dir = data_path / 'subjects'

sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample',
                                    'sample_audvis_raw.fif')

raw_path = os.path.join(sample_data_folder, 'MEG', 'sample',
                                    'sample_audvis_raw.fif')


# Data Summary and PSD

In [8]:
raw = mne.io.read_raw_fif(sample_data_raw_file, preload=True )
raw.crop(tmax=60.)


report = mne.Report(title='Raw example')
# This method also accepts a path, e.g., raw=raw_path
report.add_raw(raw=raw, title='Raw', psd=True) 
report.save('report_raw.html', overwrite=True)

Opening raw data file C:\Users\AA086655\mne_data\MNE-sample-data\MEG\sample\sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Reading 0 ... 166799  =      0.000 ...   277.714 secs...
Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Using matplotlib as 2D backend.
Effective window size : 3.410 (s)
Effective window size : 3.410 (s)
Effective window size : 3.410 (s)
Overwriting existing file.
Saving report to : C:\Users\AA086655\OneDrive - Cerner Corporation\Desktop\working codes\EEG_codes\report_raw.html


'C:\\Users\\AA086655\\OneDrive - Cerner Corporation\\Desktop\\working codes\\EEG_codes\\report_raw.html'

# Events Summary Report

In [9]:
events_path = sample_dir / 'sample_audvis_raw-eve.fif'
events = mne.find_events(raw=raw)
sfreq = raw.info['sfreq']

report = mne.Report(title='Events example')
report.add_events(events=events_path, title='Events from Path', sfreq=sfreq)
report.add_events(events=events, title='Events from "events"', sfreq=sfreq)
report.save('report_events.html', overwrite=True)

86 events found
Event IDs: [ 1  2  3  4  5 32]
Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Overwriting existing file.
Saving report to : C:\Users\AA086655\OneDrive - Cerner Corporation\Desktop\working codes\EEG_codes\report_events.html


'C:\\Users\\AA086655\\OneDrive - Cerner Corporation\\Desktop\\working codes\\EEG_codes\\report_events.html'

In [10]:
event_id = {
    'auditory/left': 1, 'auditory/right': 2, 'visual/left': 3,
    'visual/right': 4, 'face': 5, 'buttonpress': 32
}

metadata, _, _ = mne.epochs.make_metadata(
    events=events,
    event_id=event_id,
    tmin=-0.2,
    tmax=0.5,
    sfreq=raw.info['sfreq']
)
epochs = mne.Epochs(
    raw=raw, events=events, event_id=event_id, metadata=metadata
)

report = mne.Report(title='Epochs example')
report.add_epochs(epochs=epochs, title='Epochs from "epochs"')
report.save('report_epochs.html', overwrite=True)

Adding metadata with 7 columns
Replacing existing metadata with 7 columns
86 matching events found
Setting baseline interval to [-0.19979521315838786, 0.0] sec
Applying baseline correction (mode: mean)
Created an SSP operator (subspace dimension = 3)
3 projection items activated
Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Loading data for 86 events and 421 original time points ...
0 bad epochs dropped
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
    Using multitaper spectrum estimation with 7 DPSS windows
Overwriting existing file.
Saving report to : C:\Users\AA086655\OneDrive - Cerner Corporation\Desktop\working codes\EEG_codes\report_epochs.html


'C:\\Users\\AA086655\\OneDrive - Cerner Corporation\\Desktop\\working codes\\EEG_codes\\report_epochs.html'

# Evoked..

In [11]:
evoked_path = sample_dir / 'sample_audvis-ave.fif'
cov_path = sample_dir / 'sample_audvis-cov.fif'

evokeds = mne.read_evokeds(evoked_path, baseline=(None, 0))
evokeds_subset = evokeds[:2]  # The first two
for evoked in evokeds_subset:
    evoked.pick('eeg')  # just for speed of plotting

report = mne.Report(title='Evoked example')
report.add_evokeds(
    evokeds=evokeds_subset,
    titles=['evoked 1',  # Manually specify titles
            'evoked 2'],
    noise_cov=cov_path,
    n_time_points=5
)
report.save('report_evoked.html', overwrite=True)

Reading C:\Users\AA086655\mne_data\MNE-sample-data\MEG\sample\sample_audvis-ave.fif ...
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -199.80 ...     499.49 ms (Left Auditory)
        0 CTF compensation matrices available
        nave = 55 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Applying baseline correction (mode: mean)
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Found the data of interest:
        t =    -199.80 ...     499.49 ms (Right Auditory)
        0 CTF compensation matrices available
        nave = 61 - aspect type = 100
Projections have already been applied. Setting proj attribute to True.
Applying baseline

'C:\\Users\\AA086655\\OneDrive - Cerner Corporation\\Desktop\\working codes\\EEG_codes\\report_evoked.html'

# Covariance report

In [12]:
cov_path = sample_dir / 'sample_audvis-cov.fif'

report = mne.Report(title='Covariance report')
report.add_covariance(cov=cov_path, info=raw_path, title='Covariance')
report.save('report_cov.html', overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
    366 x 366 full covariance (kind = 1) found.
    Read a total of 4 projection items:
        PCA-v1 (1 x 102) active
        PCA-v2 (1 x 102) active
        PCA-v3 (1 x 102) active
        Average EEG reference (1 x 60) active
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
Computing rank from covariance with rank=None
    Using tolerance 2.5e-14 (2.2e-16 eps * 102 dim * 1.1  max singular value)
    Estimated rank (mag): 102
    MAG: rank 102 computed from 102 data channels with 0 projectors
Computing rank from covariance with rank=None
    Using tolerance 2.6e-12 (2.2e-16 eps * 204 dim * 56  max singular value)
    Estimated rank (grad): 204
    GRAD: rank 204 computed from 204 data channels with 0 project

'C:\\Users\\AA086655\\OneDrive - Cerner Corporation\\Desktop\\working codes\\EEG_codes\\report_cov.html'

# Projector report

In [13]:
ecg_proj_path = sample_dir / 'sample_audvis_ecg-proj.fif'
eog_proj_path = sample_dir / 'sample_audvis_eog-proj.fif'

report = mne.Report(title='Projectors example')
report.add_projs(info=raw_path, title='Projs from info')
report.add_projs(info=raw_path, projs=ecg_proj_path,
                 title='ECG projs from path')
report.add_projs(info=raw_path, projs=eog_proj_path,
                 title='EOG projs from path')
report.save('report_projs.html', overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
    Read a total of 6 projection items:
        ECG-planar-999--0.200-0.400-PCA-01 (1 x 203)  idle
        ECG-planar-999--0.200-0.400-PCA-02 (1 x 203)  idle
        ECG-axial-999--0.200-0.400-PCA-01 (1 x 102)  idle
        ECG-axial-999--0.200-0.400-PCA-02 (1 x 102)  idle
        ECG-eeg-999--0.200-0.400-PCA-01 (1 x 59)  idle
        ECG-eeg-999--0.200-0.400-PCA-02 (1 x 59)  idle
    Read a total of 6 projection items:
        EOG-planar-998--0.200-0.200-PCA-01 (1 x 203)  idle
        EOG-planar-998--0.200-0.200-PCA-02 (1 x 203)  idle
        EOG-axial-998--0.200-0.200-PCA-01 (1 x 102)  idle
        EOG-axial-998--0.200-0.200-PCA-02 (1 x 102)  idle
        EOG-eeg-998--0.200-0.200-PCA-01 (1 x 59)  idle
        EOG-eeg-998--0.200-0.200-PCA-02 (1 x 59)  idle
Overwriting existing file.
Saving re

'C:\\Users\\AA086655\\OneDrive - Cerner Corporation\\Desktop\\working codes\\EEG_codes\\report_projs.html'

# Independent Component Analysis report

In [14]:
ica = mne.preprocessing.ICA(
    n_components=15,  # fit 5 ICA components
    fit_params=dict(tol=0.01)  # assume very early on that ICA has converged
)

ica.fit(inst=raw)

# create epochs based on EOG events, find EOG artifacts in the data via pattern
# matching, and exclude the EOG-related ICA components
eog_epochs = mne.preprocessing.create_eog_epochs(raw=raw)
eog_components, eog_scores = ica.find_bads_eog(
    inst=eog_epochs,
    ch_name='EEG 001',  # a channel close to the eye
    threshold=1  # lower than the default threshold
)
ica.exclude = eog_components

report = mne.Report(title='ICA example')
report.add_ica(
    ica=ica,
    title='ICA cleaning',
    picks=[0, 1, 2, 3, 4,5,6,7,8,9,10,11,12,13,14],  # only plot the first two components
    inst=raw,
    eog_evoked=eog_epochs.average(),
    eog_scores=eog_scores,
    n_jobs=1  # could be increased!
)
report.save('report_ica.html', overwrite=True)

Fitting ICA to data using 364 channels (please be patient, this may take a while)
Selecting by number: 15 components
Fitting ICA took 3.9s.
Using EOG channel: EOG 061
EOG channel index for this subject is: [375]
Filtering the data to remove DC offset to help distinguish blinks from saccades
Setting up band-pass filter from 1 - 10 Hz

FIR filter parameters
---------------------
Designing a two-pass forward and reverse, zero-phase, non-causal bandpass filter:
- Windowed frequency-domain design (firwin2) method
- Hann window
- Lower passband edge: 1.00
- Lower transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 0.75 Hz)
- Upper passband edge: 10.00 Hz
- Upper transition bandwidth: 0.50 Hz (-12 dB cutoff frequency: 10.25 Hz)
- Filter length: 6007 samples (10.001 sec)

Now detecting blinks and generating corresponding events
Found 10 significant peaks
Number of EOG events detected: 10
Not setting metadata
Not setting metadata
10 matching events found
No baseline correction applied
Creat

'C:\\Users\\AA086655\\OneDrive - Cerner Corporation\\Desktop\\working codes\\EEG_codes\\report_ica.html'

# MRI & BEM Report

In [15]:
report = mne.Report(title='BEM example')
report.add_bem(
    subject='sample', subjects_dir=subjects_dir, title='MRI & BEM',
    decim=20,
    width=256
)
report.save('report_mri_and_bem.html', overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
Using surface: C:\Users\AA086655\mne_data\MNE-sample-data\subjects\sample\bem\inner_skull.surf
Using surface: C:\Users\AA086655\mne_data\MNE-sample-data\subjects\sample\bem\outer_skull.surf
Using surface: C:\Users\AA086655\mne_data\MNE-sample-data\subjects\sample\bem\outer_skin.surf
Overwriting existing file.
Saving report to : C:\Users\AA086655\OneDrive - Cerner Corporation\Desktop\working codes\EEG_codes\report_mri_and_bem.html


'C:\\Users\\AA086655\\OneDrive - Cerner Corporation\\Desktop\\working codes\\EEG_codes\\report_mri_and_bem.html'

# Coregistration Report

In [ ]:
trans_path = sample_dir / 'sample_audvis_raw-trans.fif'

report = mne.Report(title='Coregistration report')
report.add_trans(trans=trans_path, info=raw_path, subject='sample',subjects_dir=subjects_dir, title='Coregistration')
report.save('report_coregistration.html', overwrite=True)

Embedding : jquery-3.6.0.min.js
Embedding : bootstrap.bundle.min.js
Embedding : bootstrap.min.css
Embedding : highlightjs/highlight.min.js
Embedding : highlightjs/atom-one-dark-reasonable.min.css
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
Using mayavi 3d backend.



<ipython-input-16-75abf4050c43>:4: DeprecationWarning: The Mayavi 3D backend is deprecated in 0.24 and will be removed in the next version of MNE-Python. Use the pyvistaqt or notebook backends instead
  report.add_trans(trans=trans_path, info=raw_path, subject='sample',subjects_dir=subjects_dir, title='Coregistration')
